# YOLO Image detection model tutorial
This notebook contains all the steps you need to run an image detection model. The single-class remap is **optional**
so you can keep your original multi-class dataset for later experiments.

**Before running everything, what to install** (in the venv):
1. VS and CUDA (for nvidia gpu)
2. create and activate the venv
3. pip install --upgrade pip setuptools wheel
4. pip install ultralytics
5. the correct version of pytorch (https://pytorch.org/get-started/locally/)

**Sections**:
1. Config
2. Validate input layout
3. (Optional) Remap to single class → `DEST_SINGLE`
4. Split into `train/val/test` from `DATASET_FOR_SPLIT`
5. Write `data.yaml` (reads `classes.txt` if present; otherwise infers `nc`)
6. Train (Ultralytics YOLO)
7. Validate on test images


**Expected input format** (no zip needed):
```
SRC_ROOT/
  images/  (jpg, png, ...)
  labels/  (txt per image: class cx cy w h ...)
  classes.txt  (optional: one class name per line)
```


## 1) Configuration

**This will probably be the only block that needs editing**; here is where most of the decision making is done:
1. Make sure to choose the right paths
2. Decide now if you want to use single-class or not 
3. Choose the ratio between training, validate and test in the data set
4. Choose the YOLO model size you want to run; e.g. the 'm' in yolo11m stands for medium


In [ ]:
from pathlib import Path
# 1

# Set the path to the original dataset
SRC_ROOT = Path(r"Path to the original dataset")

# Where to write the optional single-class copy:
DEST_SINGLE = Path(r"Path to the single-class copy")

# Output root for the split + data.yaml:
OUT_ROOT = Path(r"Path to the output root")

# 2

# DATASET_FOR_SPLIT = SRC_ROOT  # change to DEST_SINGLE if you want a single-class copy
DATASET_FOR_SPLIT = DEST_SINGLE # change to SRC_ROOT if you don't want a single-class copy


# 3
TRAIN_RATIO, VAL_RATIO, TEST_RATIO = 0.85, 0.10, 0.05
SMALL_TEST_COUNT = None  # set None to disable and use ratios
RANDOM_SEED = 42
IMG_EXT = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

# Training options:
MODEL_WEIGHTS = "yolo11l.pt"
IMG_SIZE = 2016                 # This has to be a multiple of 32
EPOCHS = 60
BATCH = 1                       # This has to be a power of 2

SRC_ROOT, DEST_SINGLE, DATASET_FOR_SPLIT, OUT_ROOT

## 2) Validate input layout
Checks that `images/` and `labels/` exist.

In [ ]:
IMG_SRC = SRC_ROOT / "images"
LBL_SRC = SRC_ROOT / "labels"
assert IMG_SRC.exists() and IMG_SRC.is_dir(), f"Missing images folder: {IMG_SRC}"
assert LBL_SRC.exists() and LBL_SRC.is_dir(), f"Missing labels folder: {LBL_SRC}"
print("✅ Found:", IMG_SRC)
print("✅ Found:", LBL_SRC)

## 3) (Optional) Remap to single class
Run this **only if** you want a single-class dataset copy. Skip this cell to keep multiclass.

In [ ]:
from pathlib import Path
import shutil

IMG_DST = DEST_SINGLE / "images"
LBL_DST = DEST_SINGLE / "labels"
IMG_DST.mkdir(parents=True, exist_ok=True)
LBL_DST.mkdir(parents=True, exist_ok=True)

def rewrite_to_single_class(label_text: str) -> str:
    out = []
    for line in label_text.splitlines():
        line = line.strip()
        if not line:
            continue
        parts = line.split()
        if len(parts) >= 5:
            parts[0] = "0"
            out.append(" ".join(parts))
        else:
            out.append(line)
    return ("\n".join(out) + "\n") if out else ""

imgs = sorted([p for p in IMG_SRC.rglob("*") if p.suffix.lower() in IMG_EXT])
pairs = []
for im in imgs:
    lb = LBL_SRC / f"{im.stem}.txt"
    if lb.exists():
        pairs.append((im, lb))

if not pairs:
    raise SystemExit("No image/label pairs found in SRC_ROOT.")

copied = 0
for im, lb in pairs:
    shutil.copy2(im, IMG_DST / im.name)
    txt = lb.read_text(encoding="utf-8")
    remapped = rewrite_to_single_class(txt)
    (LBL_DST / lb.name).write_text(remapped, encoding="utf-8")
    copied += 1

(DEST_SINGLE / "classes.txt").write_text("colony\n", encoding="utf-8")
print(f"✅ Single-class copy complete. Pairs: {copied}")
print("Single-class dataset at:", DEST_SINGLE)
print("👉 To use it in the split, set DATASET_FOR_SPLIT = DEST_SINGLE in the Config cell.")

## 4) Split into train / val / test
Run this to split either the multi or the single class data set.

In [ ]:
from pathlib import Path
import random, shutil

random.seed(RANDOM_SEED)

IMG_SRC2 = DATASET_FOR_SPLIT / "images"
LBL_SRC2 = DATASET_FOR_SPLIT / "labels"

imgs2 = sorted([p for p in IMG_SRC2.rglob("*") if p.suffix.lower() in IMG_EXT])
pairs2 = [(im, LBL_SRC2 / f"{im.stem}.txt") for im in imgs2 if (LBL_SRC2 / f"{im.stem}.txt").exists()]
if not pairs2:
    raise SystemExit("No pairs found in DATASET_FOR_SPLIT.")

random.shuffle(pairs2)
N = len(pairs2)

if SMALL_TEST_COUNT is not None:
    test_n = min(SMALL_TEST_COUNT, N)
    rem = N - test_n
    tr = int(rem * (TRAIN_RATIO / (TRAIN_RATIO + VAL_RATIO)))
    val_n = rem - tr
    train_n = tr
else:
    test_n  = int(N * TEST_RATIO)
    train_n = int(N * TRAIN_RATIO)
    val_n   = N - train_n - test_n

train_pairs = pairs2[:train_n]
val_pairs   = pairs2[train_n:train_n+val_n]
test_pairs  = pairs2[train_n+val_n:]

OUT_IMG_T = OUT_ROOT / "train" / "images"
OUT_LBL_T = OUT_ROOT / "train" / "labels"
OUT_IMG_V = OUT_ROOT / "val" / "images"
OUT_LBL_V = OUT_ROOT / "val" / "labels"
OUT_IMG_S = OUT_ROOT / "test" / "images"
OUT_LBL_S = OUT_ROOT / "test" / "labels"

for p in [OUT_IMG_T, OUT_LBL_T, OUT_IMG_V, OUT_LBL_V, OUT_IMG_S, OUT_LBL_S]:
    p.mkdir(parents=True, exist_ok=True)

def cp_pair(im: Path, lb: Path, di: Path, dl: Path):
    shutil.copy2(im, di / im.name)
    shutil.copy2(lb, dl / f"{im.stem}.txt")

for im, lb in train_pairs: cp_pair(im, lb, OUT_IMG_T, OUT_LBL_T)
for im, lb in val_pairs:   cp_pair(im, lb, OUT_IMG_V, OUT_LBL_V)
for im, lb in test_pairs:  cp_pair(im, lb, OUT_IMG_S, OUT_LBL_S)

print(f"✅ Split done. total={N}  train={len(train_pairs)}  val={len(val_pairs)}  test={len(test_pairs)}")
print("Split output root:", OUT_ROOT)

## 5) Write `data.yaml`
- If `classes.txt` exists in already, uses it for `names` and `nc`
- Otherwise, makes a new one

In [ ]:
import re

classes_file = (DATASET_FOR_SPLIT / "classes.txt")
names = None
if classes_file.exists():
    names = [ln.strip() for ln in classes_file.read_text(encoding="utf-8").splitlines() if ln.strip()]

if names is None:
    # infer nc from labels
    max_c = 0
    for lbl in (OUT_ROOT / "train" / "labels").glob("*.txt"):
        for line in lbl.read_text(encoding="utf-8").splitlines():
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) >= 5 and parts[0].isdigit():
                cid = int(parts[0])
                if cid > max_c:
                    max_c = cid
    nc = max_c + 1
    names = [f"class_{i}" for i in range(nc)]
else:
    nc = len(names)

yaml = f'''train: "{(OUT_ROOT / 'train' / 'images').as_posix()}"
val:   "{(OUT_ROOT / 'val'   / 'images').as_posix()}"
test:  "{(OUT_ROOT / 'test'  / 'images').as_posix()}"

nc: {nc}
names: {names}
'''
(OUT_ROOT / "data.yaml").write_text(yaml, encoding="utf-8")
print("✅ Wrote:", OUT_ROOT / "data.yaml")
print("nc:", nc, "names:", names)

**Last check before training**: Make sure cuda is available and the GPU is used. 

In [ ]:
import torch
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))

## 6) Train
Trains a YOLO model with the generated `data.yaml`.

In [ ]:
from ultralytics import YOLO
data_yaml = OUT_ROOT / "data.yaml"
model = YOLO(MODEL_WEIGHTS)
model.train(data=str(data_yaml), imgsz=IMG_SIZE, epochs=EPOCHS, batch=BATCH)

## 7) Validate
Runs the latest 'best' run on the test dataset. 

In [ ]:
import os, glob, time
from pathlib import Path
from ultralytics import YOLO
from IPython.display import Image, display

DATA_YAML   = OUT_ROOT / "data.yaml"
TEST_IMAGES = OUT_ROOT / "test" / "images"

# Finds the latest 'best' weights
def latest_best_weights(root="runs/detect"):
    cand = []
    for w in Path(root).glob("train*/weights/best.pt"):
        try:
            mtime = w.stat().st_mtime
        except FileNotFoundError:
            continue
        cand.append((mtime, w))
    if not cand:
        raise FileNotFoundError(f"No best.pt found under {root}/train*/weights/")
    cand.sort(key=lambda x: x[0], reverse=True)
    return cand[0][1]

BEST_WEIGHTS = str(latest_best_weights())
print(f"✅ Using latest best weights: {BEST_WEIGHTS}")


model = YOLO(BEST_WEIGHTS)
model.predict(
    source=TEST_IMAGES,
    save=True,
    hide_labels=True,
    hide_conf=True,
    name="predict-test",
    exist_ok=True
)

metrics = model.val(
    data=DATA_YAML,
    split="test",
    plots=True,
    save_json=True,
    name="val-test",
    exist_ok=True
)

print("\n📂 Test-set evaluation artifacts saved in: runs/detect/val-test/")
print(" - confusion_matrix.png")
print(" - PR_curve.png")
print(" - F1_curve.png")
print(" - results.png")
print(" - predictions.json")
